# Episode 22 — Visualize Results

In this notebook you will learn how to visualize quantum execution results with **Qiskit**:

- Plot Sampler output distributions with `plot_histogram`.
- Make quick bar charts for Estimator expectation values (with error bars).
- Save or display figures depending on your environment.

> **Prerequisites**: You should have an IBM Quantum account configured for Qiskit Runtime, or switch to a local simulator for testing.

## Package versions
Run the following cell to see the versions of the key packages used by this notebook.

In [1]:
import sys, qiskit, qiskit_ibm_runtime, matplotlib
print("Python:", sys.version.split()[0])
print("Qiskit:", qiskit.__version__)
print("Qiskit IBM Runtime:", qiskit_ibm_runtime.__version__)
print("Matplotlib:", matplotlib.__version__)

## 1) Plot a histogram from Sampler results
We'll build a simple 2-qubit Bell state, transpile it to the target backend, run it with `SamplerV2`, and plot the measurement histogram.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit import QuantumCircuit
from qiskit.visualization import plot_histogram

# Initialize service and pick a backend (real device). For local tests: set simulator=True
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)
print("Backend:", backend.name)

# Build Bell state
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# Transpile to ISA for the backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)

# Run with Sampler
sampler = Sampler(mode=backend)
job = sampler.run([isa_bell])
result = job.result()
print(result)

# Plot histogram of counts
fig = plot_histogram(result[0].data.meas.get_counts())
fig

### Customize the histogram
You can compare multiple runs, sort bars, set figure size, annotate bars, and so on.

In [ ]:
# Change default shots for a second run
sampler.options.default_shots = 1000
job2 = sampler.run([isa_bell])
res2 = job2.result()

fig2 = plot_histogram(
    [
        result[0].data.meas.get_counts(),
        res2[0].data.meas.get_counts(),
    ],
    legend=["first", "second"],
    sort="desc",
    figsize=(12, 8),
    color=["orange", "black"],
    bar_labels=False,
)
fig2

### Save or show figures
If you're running outside Jupyter, explicitly call `.show()` (for interactive backends) or `.savefig("filename.png")` to write to disk.

In [ ]:
# Example: save last histogram to a PNG file
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
outfile = f"histogram_{timestamp}.png"
fig2.savefig(outfile)
print("Saved:", outfile)

## 2) Visualize Estimator expectation values
Qiskit doesn't include a pre-built plot for Estimator values, but Matplotlib makes this simple. We'll estimate EVs for several Pauli observables and then draw a bar chart with error bars.

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler import generate_preset_pass_manager
from matplotlib import pyplot as plt

# Prepare a small circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.crx(1.5, 0, 1)

labels = ["ZZ", "XX", "YZ", "ZY", "XY", "XZ", "ZX"]
ops = [SparsePauliOp(lbl) for lbl in labels]

pm2 = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm2.run(qc)
isa_ops = [op.apply_layout(isa_qc.layout) for op in ops]

# Reshape so broadcasting matches the primitive input expectations
ops_array = np.fromiter(isa_ops, dtype=object).reshape((len(isa_ops), 1))

est = Estimator(mode=backend)
job_e = est.run([(isa_qc, ops_array)])
pub = job_e.result()[0]
evs = pub.data.evs.flatten()
errs = pub.data.stds.flatten()
print(pub)

# Bar chart with error bars
fig3, ax = plt.subplots()
ax.bar(labels, evs, yerr=errs, capsize=2)
ax.set_title("Expectation values (with standard errors)")
fig3

## 3) (Optional) Configure error mitigation for Estimator
You can trade off accuracy and runtime by setting `resilience_level` or turning specific mitigation options on/off.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

est_mitigated = Estimator(mode=backend, options={"resilience_level": 2})
job_em = est_mitigated.run([(isa_qc, ops_array)])
pub_m = job_em.result()[0]
print("Mitigated EVs:", pub_m.data.evs.flatten())

## Additional information

**Created by:** Ricard Santiago Raigada García

**Version:** 1.0.0